Function Calling

<img src="Images/ima.png" width="800" height="400">

Library

In [78]:
import os
import json
import requests
from dotenv import load_dotenv
from openai import  AzureOpenAI

Load .ENV

In [91]:
load_dotenv(dotenv_path='/Users/veronica/IA_2024/Modulo02_Sprint02/my-python-project/.venv/.env', override=True)
az_openai_client=os.getenv("AZURE_OPENAI_API_KEY")
az_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL")
az_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
az_openai_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME ")
bing_api_key = os.getenv("BING_API_KEY")

print(az_openai_endpoint)

None


Crear Cliente Azure Open AI

In [103]:
#
az_openai_client = AzureOpenAI(
	azure_endpoint= az_openai_endpoint,
	api_key=az_openai_api_key,
	api_version="2024-05-01-preview",
)
message = []

Functions

In [116]:
def store_message_in_list(message_list, message_role, **kwargs):
	# Create the base message with the required role
	message = {"role": message_role}
	# Dynamically add additional fields from kwargs
	message.update(kwargs)
	# Append the message to the list
	message_list.append(message)

# Azure OpenAI API Calls
def call_azure_openai(az_openai_client, az_openai_deployment_name, messages):
	# Build the parameters dynamically
	params = {
		"model": az_openai_deployment_name,
		"messages": messages
	}
	# Call the Azure OpenAI API
	response = az_openai_client.chat.completions.create(**params)
	return response

def send_user_input_to_az_openai(user_input):
    """
    Envía la entrada del usuario al servicio Azure OpenAI y devuelve la respuesta.

    :param user_input: Texto ingresado por el usuario (str)
    :return: Respuesta del modelo (str)
    """
    try:
        # Crear cliente OpenAI con Azure
        client = OpenAIClient(
            endpoint=AZURE_OPENAI_ENDPOINT,
            credential=AzureKeyCredential(AZURE_OPENAI_API_KEY)
        )

        # Configuración de la solicitud
        response = client.get_chat_completions(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "Eres un asistente útil."},  # Contexto inicial
                {"role": "user", "content": user_input}  # Entrada del usuario
            ],
            temperature=0.7,  # Control de creatividad
            max_tokens=200  # Longitud máxima de la respuesta
        )

        # Extraer la respuesta del modelo
        model_reply = response.choices[0].message["content"]
        return model_reply

    except Exception as e:
        return f"Error al comunicarse con Azure OpenAI: {str(e)}"

Inicializar Mensajes

In [106]:
SYSTEM_MESSAGE="Soy un bot de Azure OpenIA , Y me gustaria ayudarte en lo que necesites referente a la organizacion de vuelos"
store_message_in_list(message, "system", content=SYSTEM_MESSAGE)

print (SYSTEM_MESSAGE)

prueba


Comprobar Fecha cut-off del modelo

In [123]:
user_input = "Hasta que fecha has aprendido?"

response = send_user_input_to_az_openai(user_input)

print("Respuesta del modelo:", response)

Respuesta del modelo: Error al comunicarse con Azure OpenAI: name 'OpenAIClient' is not defined


Busqueda en Bing

In [59]:
import requests
# Lista para almacenar los mensajes
messages = []

def store_message_in_list(role, content):
    message = {"role": role, "content": content}
    messages.append(message)
def search_bing_for_data(query):
    headers = {
        'Ocp-Apim-Subscription-Key': bing_api_key
    }
    params = {
        'q': query,
        'count': 1
    }
    response = requests.get('https://api.bing.microsoft.com/v7.0/search', headers=headers, params=params)
    response.raise_for_status()
    return response.json()

# Realizar una búsqueda de ejemplo
query = "inteligencia artificial"
results = search_bing_for_data(query)

# Imprimir los resultados
for result in results['webPages']['value']:
    print(result['name'])
    print(result['url'])
    print(result['snippet'])
    print()

OpenAI
https://openai.com/
Bring your imagination to life from text, image, or video. A new series of AI models designed to spend more time thinking before they respond. Chat about email, screenshots, files, and anything on your screen. Instant answers. Greater productivity. Endless inspiration.



In [60]:
search_bing_for_data("Capital de Francia")

{'_type': 'SearchResponse',
 'queryContext': {'originalQuery': 'Capital de Francia'},
 'webPages': {'webSearchUrl': 'https://www.bing.com/search?q=Capital+de+Francia',
  'totalEstimatedMatches': 12600000,
  'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
    'name': 'París - Wikipedia, la enciclopedia libre',
    'url': 'https://es.wikipedia.org/wiki/Par%C3%ADs',
    'thumbnailUrl': 'https://www.bing.com/th?id=OIP.ydQwflIykYmIBA4N_ymevgHaEo&w=80&h=80&c=1&pid=5.1',
    'isFamilyFriendly': True,
    'displayUrl': 'https://es.wikipedia.org/wiki/París',
    'snippet': 'París es la capital de Francia y su ciudad más poblada, con una rica historia y cultura. Conoce su origen, su geografía, su economía, sus monumentos y su vida social en este artículo de Wikipedia.',
    'dateLastCrawled': '2025-01-19T12:22:00.0000000Z',
    'primaryImageOfPage': {'thumbnailUrl': 'https://www.bing.com/th?id=OIP.ydQwflIykYmIBA4N_ymevgHaEo&w=80&h=80&c=1&pid=5.1',
     'width': 88,
     'he

Registro de Tool para llamadas Azure Open AI

In [61]:
tool_map = {
      "search_bing_for_data": search_bing_for_data
},
tools = [
     {
            "type": "function",
            "function": {
                  "name": "search_bing_for_data",
                  "description": "Search for data on the internet for a given topic",
                  "parameters": {
                        "type": "object",
                        "properties": {
                              "query": {
                              "type": "string",
                              "description": "The content to search for",
                              "required": ["query"]
                              }
                        }
                  }
                  }
            }
      
]

Llamada API con tool

In [62]:
from pyexpat.errors import messages


    
def call_azure_openai_with_tools(az_openai_client, az_openai_deployment_name, messages, tools, tool_choice="auto"):
    # Build the parameters dynamically
    params = {
        "model": az_openai_deployment_name,
        "messages": messages,
        "tools": tools,
        "tool_choice": tool_choice
    }
    # Call the Azure OpenAI API
    response = az_openai_client.chat.completions.create(**params)
    return response.choices[0]

def send_user_input_to_az_openai_with_tools(
        user_input,
        az_openai_client=az_openai_client,
        az_openai_deployment_name=az_openai_deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto"
     
):
    
       
    
    # Store user message
    store_message_in_list(messages, "user", content=user_input)
    # Query Azure OpenAI
    az_openai_response = call_azure_openai_with_tools(az_openai_client, az_openai_deployment_name, messages, tools, tool_choice)
    
    az_openai_response
    
    return az_openai_response



La respuesta de un modelo indica  que hay que ejecutar una Tool

In [ ]:
post_cutoff_question = "quien gano la- EURO 2024?'"

model_response = send_user_input_to_az_openai_with_tools(post_cutoff_question)
print(model_response)